# Building the Pipeline on Local Spark

1. Setting up
2. Helpers
3. Pull stock info
4. Pull short interests
5. Pull stock prices
6. Combine datasets

## 1. Setting Up

We want to have a logging feature that works for both Jupyter notebook and Spark environments.

1. As it turned out, Spark has "WARN" but does not have "WARNING" level, while in current Python (3.6.x), "WARN" is deprecated, "WARNING" should be used instead.
2. Therefore, we create a custom "WARN" level as well as function `logger.warn` for Jupyter notebook.
3. As shown in [this StackOverflow post](https://stackoverflow.com/questions/35326814/change-level-logged-to-ipython-jupyter-notebook), this is not straightforward due to a Jupyter notebook bug. We need to workaround this by specifying an invalid value first, which we do in the code cell below.

In [1]:

# Run this, but don't copy into etl scripts
# workaround via specifying an invalid value first
%config Application.log_level='WORKAROUND'
import logging
logging.WARN = 21
logging.addLevelName(logging.WARN, 'WARN')

def warn(self, message, *args, **kws):
    if self.isEnabledFor(logging.WARN):
        # Yes, logger takes its '*args' as 'args'.
        self._log(logging.WARN, message, args, **kws) 
logging.Logger.warn = warn


logger = logging.getLogger()
logger.setLevel(logging.WARN)
logger.warn('hello')

# ------------------
from pyspark.sql import SparkSession

spark = spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
#         .config("spark.eventLog.enabled", "true") \
#         .config("spark.eventLog.dir" "test_data/spark-logs") \

import pandas as pd
import configparser
config = configparser.ConfigParser()
config.read('airflow/config.cfg')
import tqdm

ERROR:root:The 'log_level' trait of an IPKernelApp instance must be any of (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL'), but a value of 'WORKAROUND' <class 'str'> was specified.
WARN:root:hello


['airflow/config.cfg']

In [2]:
# Run this cell, and also copy to all etl scripts, or simply include in common.py

import requests
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row

from py4j.java_gateway import java_import

### Test

In [75]:
delete_path(spark, 's3a://short-interest-effect', '/data/raw/stock_info_nasdaq')

## 2. Helpers

Include this code as helpers in all next etl scripts.

### Code

In [3]:
AWS_ACCESS_KEY_ID = config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [46]:
from py4j.protocol import Py4JJavaError

sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", AWS_ACCESS_KEY_ID)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)


def delete_path(spark, host, path):
    sc = spark.sparkContext
    java_import(sc._gateway.jvm, "java.net.URI")
    uri = sc._gateway.jvm.java.net.URI
    fs = (sc._jvm.org
          .apache.hadoop
          .fs.FileSystem
          .get(uri(host), sc._jsc.hadoopConfiguration())
          )
    fs.delete(sc._jvm.org.apache.hadoop.fs.Path(host+path), True)
    
    
def copyMerge(spark, host, src_dir, dst_file, overwrite=False, deleteSource=False, debug=False):
    
    sc = spark.sparkContext
    
    hadoop = sc._jvm.org.apache.hadoop
#     conf = hadoop.conf.Configuration()
    conf = sc._jsc.hadoopConfiguration()
#     fs = hadoop.fs.FileSystem.get(conf)
    java_import(sc._gateway.jvm, "java.net.URI")
    uri = sc._gateway.jvm.java.net.URI
    fs = (sc._jvm.org
          .apache.hadoop
          .fs.FileSystem
          .get(uri(host), sc._jsc.hadoopConfiguration())
          )

    # check files that will be merged
    files = []
    for f in fs.listStatus(hadoop.fs.Path(src_dir)):
        if f.isFile():
            files.append(f.getPath())
    if not files:
        raise ValueError("Source directory {} is empty".format(src_dir))
    files.sort(key=lambda f: str(f))

    # dst_permission = hadoop.fs.permission.FsPermission.valueOf(permission)      # , permission='-rw-r-----'
    out_stream = fs.create(hadoop.fs.Path(dst_file), overwrite)

    try:
        # loop over files in alphabetical order and append them one by one to the target file
        for file in files:
            if debug: 
                print("Appending file {} into {}".format(file, dst_file))

            in_stream = fs.open(file)   # InputStream object
            try:
                hadoop.io.IOUtils.copyBytes(in_stream, out_stream, conf, False)     # False means don't close out_stream
            finally:
                in_stream.close()
    finally:
        out_stream.close()

    if deleteSource:
        fs.delete(hadoop.fs.Path(src_dir), True)    # True=recursive
        if debug:
            print("Source directory {} removed.".format(src_dir))
            
            
def spark_table_exists(host, table_path):
    URI           = sc._gateway.jvm.java.net.URI
    Path          = sc._gateway.jvm.org.apache.hadoop.fs.Path
    FileSystem    = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
    # Configuration = sc._gateway.jvm.org.apache.hadoop.conf.Configuration
    Configuration = sc._jsc.hadoopConfiguration


    fs = FileSystem.get(URI(host), Configuration())

    try:
        status = fs.listStatus(Path(table_path))

        return True
    except Py4JJavaError as e:
        if 'FileNotFoundException' in str(e):
            return False
        else:
            print(e)
    
    
def check_basic_quality(logger, host, table_path, table_type='csv'):
    """ Checks quality of DAG.
    
    We do this by checking if the table exists and is not empty.
    
    Args:
        - table_type(str): 'parquet' or 'csv'
    """
    if not spark_table_exists(host, table_path):
        logger.warn("(FAIL) Table {} does not exist".format(host+table_path))
    else:
        if table_type == 'parquet':
            count = spark.read.parquet(host+table_path).count()
        elif table_type == 'csv':
            count = spark.read.csv(host+table_path, header=True).count()
            
        if count == 0:
            logger.warn("(FAIL) Table {} is empty.".format(host+table_path))
        else:
            logger.warn("(SUCCESS) Table {} has {} rows.".format(host+table_path, count))

### Test

In [5]:
print(spark_table_exists('s3a://short-interest-effect', 'data/test_table')) # Fails due to lack of '/' before the table path
print(spark_table_exists('s3a://short-interest-effect', '/data/test_table'))
print(spark_table_exists('', 'test_data/test_table'))

check_basic_quality(logger, 's3a://short-interest-effect', '/data/test_table')

False
False
True


WARN:root:(FAIL) Table s3a://short-interest-effect/data/test_table does not exist


## 3. Pull Stock Info

### Code

In [6]:
# Pass into `args` argument

URL_NASDAQ = 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download'
URL_NYSE = 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download'

DB_HOST = ''
# Table names: update to add '/' in the final code.
TABLE_STOCK_INFO_NASDAQ = 'test_data/raw/stock_info_nasdaq'
TABLE_STOCK_INFO_NYSE = 'test_data/raw/stock_info_nyse'

In [7]:
def pull_stock_info(url, db_host, table_path):
    response = requests.get(url)
    if response.status_code == 200 or response.status_code == 201:
        content = response.content.decode('utf-8')
        content = content.replace('Summary Quote', 'SummaryQuote')
        delete_path(spark, db_host, table_path)
        df = spark.createDataFrame([[content]], ['info_csv'])
        df.rdd.map(lambda x: x['info_csv'].replace("[","").replace("]", "")).saveAsTextFile(db_host+table_path)
        logger.warn("Stored data from {} to {}".format(url, db_host+table_path))
    else:
        logger.warn("Failed to connect to {}. We will use existing stock info data if they have been created.".format(url))
        
    
pull_stock_info(URL_NASDAQ, DB_HOST, TABLE_STOCK_INFO_NASDAQ)
pull_stock_info(URL_NYSE, DB_HOST, TABLE_STOCK_INFO_NYSE)

WARN:root:Stored data from https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download to test_data/raw/stock_info_nasdaq
WARN:root:Stored data from https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download to test_data/raw/stock_info_nyse


### Test

In [8]:
df = spark.read.csv(DB_HOST+TABLE_STOCK_INFO_NASDAQ, header=True, inferSchema=True) \
               .drop('_c8').toPandas()
df.describe()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
count,3583,3583,3583,3583,3583,3583,3583,3583
unique,3583,3153,2855,2828,44,13,128,3583
top,SOHOB,Credit Suisse AG,25.75,n/a,n/a,Health Care,n/a,https://old.nasdaq.com/symbol/oxbrw
freq,1,10,7,310,1891,782,524,1


In [9]:
df = spark.read.csv(DB_HOST+TABLE_STOCK_INFO_NYSE,
                    header=True, ignoreLeadingWhiteSpace=True, inferSchema=True) \
               .drop('_c8').toPandas()
df.describe()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
count,3092,3092,3092,3092,3092,3092,3092,3092
unique,3092,2438,2417,1922,36,13,133,3092
top,CNR,Bank of America Corporation,n/a,n/a,n/a,n/a,n/a,https://old.nasdaq.com/symbol/rio
freq,1,14,105,702,1659,1011,1011,1


In [10]:
df.head(5)

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,DDD,3D Systems Corporation,11.38,$1.35B,n/a,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd
1,MMM,3M Company,178.15,$102.45B,n/a,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm
2,WBAI,500.com Limited,7.41,$318.62M,2013,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai
3,WUBA,58.com Inc.,62.59,$9.36B,2013,Technology,"Computer Software: Programming, Data Processing",https://old.nasdaq.com/symbol/wuba
4,EGHT,8x8 Inc,19.34,$1.94B,n/a,Technology,EDP Services,https://old.nasdaq.com/symbol/eght


### Quality-check

In [11]:
check_basic_quality(logger, DB_HOST, TABLE_STOCK_INFO_NASDAQ, table_type='csv')
check_basic_quality(logger, DB_HOST, TABLE_STOCK_INFO_NYSE, table_type='csv')

WARN:root:(SUCCESS) Table test_data/raw/stock_info_nasdaq has 3583 rows.
WARN:root:(SUCCESS) Table test_data/raw/stock_info_nyse has 3092 rows.


## 4. Pull Short Interest

#### Parallelize based on stocks or parallelize based on returned data points?

At the time of writing (2020-01-15), we have 3582 stocks from NASDAQ and 3092 stocks from NYSE. The earliest date is 2013-04-01, which accounts for nearly 1700 data points (261 working days each year).

For each stock, we will need to connect to an external API (Quandl or QuoteMedia). This will take more of the processing time rather than data processing. Therefore, we parallelize based on the stocks rather than returned data points. This way, multiple Spark nodes can connect to different URLs and pull the data. The downside is, obviously, for each node we will have to iteratively process the data, but this is still faster as there are fewer data points than the stocks, at least until several years in the future (There might be a solution that allows each spark node to parallelize...).

### Code

In [12]:
# exchange_map = {
#     'nasdaq': 'FNSQ',
#     'nyse': 'FNYX'
# }

In [13]:
url = "https://www.quandl.com/api/v3/datasets/FINRA/FNYX_FB?api_key={}".format(config['Quandl']['API_KEY'])
result = requests.get(url).json()
print(result['dataset']['data'][0])
print(result['dataset']['column_names'])
col_names = [result['dataset']['column_names']] * len(result['dataset']['data'])
newdata = []
for i, cols in enumerate(col_names):
    newdata.append(dict(zip(cols, result['dataset']['data'][i])))
newdata[:2]

['2020-01-24', 626537.0, 0.0, 1618628.0]
['Date', 'ShortVolume', 'ShortExemptVolume', 'TotalVolume']


[{'Date': '2020-01-24',
  'ShortVolume': 626537.0,
  'ShortExemptVolume': 0.0,
  'TotalVolume': 1618628.0},
 {'Date': '2020-01-23',
  'ShortVolume': 351564.0,
  'ShortExemptVolume': 0.0,
  'TotalVolume': 998258.0}]

#### Write to single tables

In [84]:
# Pass into `args` argument

START_DATE = config['App']['START_DATE']
QUANDL_API_KEY = config['Quandl']['API_KEY']
YESTERDAY_DATE = '2019-12-16'
LIMIT = 1
# STOCKS = ['FB', 'GOOG', 'AMZN', 'TRMT', 'TSLA', 'MCD', 'NFLX']
STOCKS = []
AWS_ACCESS_KEY_ID = config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['AWS']['AWS_SECRET_ACCESS_KEY']
DB_HOST = ''

# Table names: update to add '/' in the final code.
TABLE_STOCK_INFO_NASDAQ = 'test_data/raw/stock_info_nasdaq'
TABLE_STOCK_INFO_NYSE = 'test_data/raw/stock_info_nyse'
TABLE_SHORT_INTERESTS_NASDAQ = 'test_data/raw/short_interests_nasdaq' 
TABLE_SHORT_INTERESTS_NYSE = 'test_data/raw/short_interests_nyse'

In [93]:
# %%timeit -n 1 -r 1

from datetime import datetime

def a_before_b(a, b):
    date_format = "%Y-%m-%d"

    # create datetime objects from the strings
    da = datetime.strptime(a, date_format)
    db = datetime.strptime(b, date_format)

    if da < db:
        return True
    else:
        return False
    
def convert_data(olddata, symbol, url):
    col_names = olddata['dataset']['column_names']
    col_names.append('Symbol')
    col_names.append('SourceURL')
    col_names_multiplied = [col_names] * len(olddata['dataset']['data'])
    newdata = []
    for i, cols in enumerate(col_names_multiplied):
        datum = olddata['dataset']['data'][i]
        datum.append(symbol)
        datum.append(url)
        newdata.append(dict(zip(cols, datum)))
    return newdata


def pull_short_interests(exchange, host, info_table_path, short_interests_table_path, log_every_n=100):
        
    def pull_exchange_short_interests_by_symbol(symbol, start_date, end_date):
        """
        Return:
            list of dicts [{'colname': value, ...}, ...]
        """
        url = 'https://www.quandl.com/api/v3/datasets/FINRA/'+exchange+'_{}?start_date='+start_date+'&end_date='+end_date+'&api_key='+QUANDL_API_KEY
        url = url.format(symbol)
        response = requests.get(url)
        newdata = []
        if response.status_code in [200, 201]:
            newdata = convert_data(response.json(), symbol, url)
        return newdata

    # Prepare list of stocks
    if STOCKS is not None and len(STOCKS) > 0:
        rdd1 = spark.sparkContext.parallelize(STOCKS)
        row_rdd = rdd1.map(lambda x: Row(x))
        df = spark.createDataFrame(row_rdd,['Symbol'])
    else:
        df = spark.read.csv(host+info_table_path, header=True)
        if LIMIT is not None:
            df = df.limit(LIMIT)
    symbols = df.select('Symbol').rdd.map(lambda r: r['Symbol']).collect()
    
    table_exists = spark_table_exists(host, short_interests_table_path)
    if table_exists:
        short_sdf = spark.read.csv(host+short_interests_table_path, header=True)
        
    total_rows = 0
    for i, symbol in enumerate(symbols):
        if table_exists:
            # Get the last date of a stock. If this last date >= YESTERDAY_DATE, don't do anything.
            dates = short_sdf.select('Date').where((short_sdf.Symbol == F.lit(symbol))).orderBy(F.desc('Date')).limit(1) \
                .rdd.map(lambda r: r['Date']).collect()
            if len(dates) > 0:
                if a_before_b(dates[0], YESTERDAY_DATE):
                    data = pull_exchange_short_interests_by_symbol(symbol, dates[0], YESTERDAY_DATE)
                else:
                    data = []
            else:
                data = pull_exchange_short_interests_by_symbol(symbol, START_DATE, YESTERDAY_DATE)
        else:
            data = pull_exchange_short_interests_by_symbol(symbol, START_DATE, YESTERDAY_DATE)
        total_rows += len(data)
        if (i%log_every_n == 0):
            logger.warn("{}/{} - total rows in this batch: {}".format(i+1, len(symbols), total_rows))
        
        if len(data) > 0:
            short_sdf = spark.createDataFrame(data)
            short_sdf.write.mode('append').format('csv').save(host+short_interests_table_path, header=True)
    logger.warn("done!")

pull_short_interests('FNSQ', DB_HOST, TABLE_STOCK_INFO_NASDAQ, TABLE_SHORT_INTERESTS_NASDAQ)
pull_short_interests('FNYX', DB_HOST, TABLE_STOCK_INFO_NYSE, TABLE_SHORT_INTERESTS_NYSE)

WARN:root:1/1 - total rows in this batch: 0
WARN:root:done!
WARN:root:1/1 - total rows in this batch: 0
WARN:root:done!


### Test

In [42]:
sdf = spark.read.csv(DB_HOST+TABLE_SHORT_INTERESTS_NASDAQ, header=True).limit(5).dropDuplicates(['Date', 'Symbol'])
sdf = sdf.orderBy(sdf.Date.desc())
print(sdf.count())
df = sdf.toPandas()
print(df['SourceURL'][0])
df.head(5)

5
https://www.quandl.com/api/v3/datasets/FINRA/FNSQ_AGNC?start_date=2013-04-01&end_date=2019-12-12&api_key=W_R9Vr9k9D5MDsXKRY3e


,Date,ShortExemptVolume,ShortVolume,SourceURL,Symbol,TotalVolume
0,2014-02-05,2779.0,1370782.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,3674388.0
1,2014-02-04,100.0,1785639.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,5315228.0
2,2014-02-03,400.0,1190818.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,4074361.0
3,2014-01-31,123.0,672564.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,1936362.0
4,2014-01-30,0.0,492736.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,1778221.0


In [43]:
df.sort_values(by=['Date', 'Symbol'], ascending=True).head(5)

,Date,ShortExemptVolume,ShortVolume,SourceURL,Symbol,TotalVolume
4,2014-01-30,0.0,492736.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,1778221.0
3,2014-01-31,123.0,672564.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,1936362.0
2,2014-02-03,400.0,1190818.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,4074361.0
1,2014-02-04,100.0,1785639.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,5315228.0
0,2014-02-05,2779.0,1370782.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,AGNC,3674388.0


### Quality-check

In [44]:
# Pass into `args` argument

STOCKS = ['FB', 'GOOG', 'AMZN', 'TRMT', 'TSLA', 'MCD', 'NFLX']
DB_HOST = ''

# Table names: update to add '/' in the final code.
TABLE_STOCK_INFO_NASDAQ = 'test_data/raw/stock_info_nasdaq'
TABLE_STOCK_INFO_NYSE = 'test_data/raw/stock_info_nyse'
TABLE_SHORT_INTERESTS_NASDAQ = 'test_data/raw/short_interests_nasdaq' 
TABLE_SHORT_INTERESTS_NYSE = 'test_data/raw/short_interests_nyse'

In [47]:
if STOCKS is None or len(STOCKS) == 0:
    check_basic_quality(logger, DB_HOST, TABLE_STOCK_INFO_NASDAQ, table_type='csv')
    check_basic_quality(logger, DB_HOST, TABLE_STOCK_INFO_NYSE, table_type='csv')
check_basic_quality(logger, DB_HOST, TABLE_SHORT_INTERESTS_NASDAQ)
check_basic_quality(logger, DB_HOST, TABLE_SHORT_INTERESTS_NYSE)

WARN:root:(SUCCESS) Table test_data/raw/short_interests_nasdaq has 528633 rows.
WARN:root:(SUCCESS) Table test_data/raw/short_interests_nyse has 483503 rows.


## 5. Pull Stock Prices

### Code (1)

In [52]:
# Pass into `args` argument

START_DATE = config['App']['START_DATE']
QUANDL_API_KEY = config['Quandl']['API_KEY']
YESTERDAY_DATE = '2019-12-12'
LIMIT = 100
STOCKS = []
AWS_ACCESS_KEY_ID = config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['AWS']['AWS_SECRET_ACCESS_KEY']
DB_HOST = ''

# Table names: update to add '/' in the final code.
TABLE_STOCK_INFO_NASDAQ = 'test_data/raw/stock_info_nasdaq'
TABLE_STOCK_INFO_NYSE = 'test_data/raw/stock_info_nyse'
TABLE_STOCK_PRICES = 'test_data/raw/prices'

URL = """http://app.quotemedia.com/quotetools/getHistoryDownload.csv?&webmasterId=501&startDay={sd}&startMonth={sm}&startYear={sy}&endDay={ed}&endMonth={em}&endYear={ey}&isRanged=true&symbol={sym}"""

In [56]:
%%timeit -n 1 -r 1

START_DAY = START_DATE.split('-')[2]
# In QuoteMedia, months start from 0, so we adjust this variable.
START_MONTH = int(START_DATE.split('-')[1]) - 1
START_YEAR = START_DATE.split('-')[0]

YST_DAY = YESTERDAY_DATE.split('-')[2]
# In QuoteMedia, months start from 0, so we adjust this variable.
YST_MONTH = int(YESTERDAY_DATE.split('-')[1]) - 1
YST_YEAR = YESTERDAY_DATE.split('-')[0]

create_table = not(spark_table_exists(DB_HOST, TABLE_STOCK_PRICES))
def pull_prices_by_symbol(symbol):
    """
    Return:
        list of dicts [{'colname': value, ...}, ...]
    """
    if create_table == True:
        # If table does not exist, pull all data.
        url = URL.format(sd=START_DAY, sm=START_MONTH, sy=START_YEAR,
                         ed=YST_DAY, em=YST_MONTH, ey=YST_YEAR,
                         sym=symbol)
    else:
        # If table had existed, pull yesterday's data.
        url = URL.format(sd=YST_DAY, sm=YST_MONTH, sy=YST_YEAR,
                         ed=YST_DAY, em=YST_MONTH, ey=YST_YEAR,
                         sym=symbol)
        
    # Code for always overwrite without temp table
#     url = URL.format(sd=START_DAY, sm=START_MONTH, sy=START_YEAR,
#                      ed=YST_DAY, em=YST_MONTH, ey=YST_YEAR,
#                      sym=symbol)

    response = requests.get(url)
    newdata = ""
    if response.status_code in [200, 201]:
        newdata = response.content.decode('utf-8')
        newdata = newdata.replace('\n', ','+symbol+'\n')
        newdata = newdata.replace('tradevol,'+symbol+'\n', 'tradevol,symbol\n')
    return newdata

schema = T.StringType()
udf_pull_prices = F.udf(pull_prices_by_symbol, schema)
    
# Prepare list of stocks
if STOCKS is not None and len(STOCKS) > 0:
    rdd1 = spark.sparkContext.parallelize(STOCKS)
    row_rdd = rdd1.map(lambda x: Row(x))
    df = spark.createDataFrame(row_rdd,['Symbol'])
else:
    df = spark.read.csv([DB_HOST+TABLE_STOCK_INFO_NASDAQ,
                        DB_HOST+TABLE_STOCK_INFO_NYSE], header=True) \
         .select('Symbol').dropDuplicates()
    if LIMIT is not None:
        df = df.limit(LIMIT)

df = df.withColumn('prices_csv', udf_pull_prices('Symbol'))

df = df.select('prices_csv').where(df['prices_csv'] != '')

table_name = DB_HOST+TABLE_STOCK_PRICES
mode = 'overwrite'
if create_table:
    logger.warn("Creating table {}".format(table_name))    
else:
    logger.warn("Appending to table {}".format(table_name))
    mode = 'append'

# Repartition here is important so we may end up with multiple CSV-like files.
# Without repartition, the headers are going to be written multiple times
# in a single csv file.
tempdir = DB_HOST+TABLE_STOCK_PRICES+'-temp'
logger.warn("    Creating temporary table {}".format(tempdir))

numrows = df.count()
df \
    .repartition(numrows).write.mode('overwrite') \
    .csv(tempdir, header=False, quote=" ")


if create_table:
    logger.warn("    done! Now creating table {}".format(table_name))
else:
    logger.warn("    done! Now appending to table {}".format(table_name))

spark.read.csv(tempdir, header=True, ignoreLeadingWhiteSpace=True, inferSchema=True) \
.write.mode(mode).csv(table_name, header=True)
# .write.mode(mode).parquet(DB_HOST+TABLE_STOCK_PRICES)


logger.warn("done!")

WARN:root:HELLO1
WARN:root:HELLO2
WARN:root:Appending to table test_data/raw/prices
WARN:root:    Creating temporary table test_data/raw/prices-temp
WARN:root:    done! Now appending to table test_data/raw/prices
WARN:root:done!


3min 3s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
# # %%timeit -n 1 -r 1

# import csv

# log_every_n = 100

# def pull_prices_by_symbol(symbol, start_date, end_date):
#     """
#     Return:
#         list of dicts [{'colname': value, ...}, ...]
#     """
    
#     sday = start_date.split('-')[2]
#     # In QuoteMedia, months start from 0, so we adjust this variable.
#     smonth = int(start_date.split('-')[1]) - 1
#     syear = start_date.split('-')[0]

#     eday = end_date.split('-')[2]
#     # In QuoteMedia, months start from 0, so we adjust this variable.
#     emonth = int(end_date.split('-')[1]) - 1
#     eyear = end_date.split('-')[0]

#     url = URL.format(sd=sday, sm=smonth, sy=syear,
#                      ed=eday, em=emonth, ey=eyear,
#                      sym=symbol)
    
#     response = requests.get(url)
#     newdata = ""
#     if response.status_code in [200, 201]:
#         newdata = response.content.decode('utf-8')
#         newdata = newdata.replace('\n', ','+symbol+'\n')
#         newdata = newdata.replace('tradevol,'+symbol+'\n', 'tradevol,symbol\n')
        
#     reader = csv.reader(scsv.split('\n'), delimiter=',')
#     for row in reader:
#         print('\t'.join(row))
        
#     return newdata

    
# # Prepare list of stocks
# if STOCKS is not None and len(STOCKS) > 0:
#     rdd1 = spark.sparkContext.parallelize(STOCKS)
#     row_rdd = rdd1.map(lambda x: Row(x))
#     df = spark.createDataFrame(row_rdd,['Symbol'])
# else:
#     df = spark.read.csv([DB_HOST+TABLE_STOCK_INFO_NASDAQ,
#                          DB_HOST+TABLE_STOCK_INFO_NYSE], header=True) \
#          .select('Symbol').dropDuplicates()
#     if LIMIT is not None:
#         df = df.limit(LIMIT)

# symbols = df.select('Symbol').rdd.map(lambda r: r['Symbol']).collect()
    
# table_exists = spark_table_exists(DB_HOST, TABLE_STOCK_PRICES)
# if table_exists:
#     price_sdf = spark.read.csv(DB_HOST+TABLE_STOCK_PRICES, header=True)

# total_rows = 0
# for i, symbol in enumerate(symbols):
#     if table_exists:
#         # Get the last date of a stock. If this last date >= YESTERDAY_DATE, don't do anything.
#         dates = price_sdf.select('date').where((price_sdf.symbol == F.lit(symbol))).orderBy(F.desc('date')).limit(1) \
#             .rdd.map(lambda r: r['date']).collect()
#         if len(dates) > 0:
#             strdata = pull_prices_by_symbol(symbol, dates[0], YESTERDAY_DATE)
#         else:
#             strdata = pull_prices_by_symbol(symbol, START_DATE, YESTERDAY_DATE)
#     else:
#         strdata = pull_prices_by_symbol(symbol, START_DATE, YESTERDAY_DATE)
#     if strdata != '':
#         sdf = spark.sparkContext.parallelize(strdata).toDF()
#         sdf.toD
# #     total_rows += len(data)
# #     if (i%log_every_n == 0):
# #         logger.warn("{}/{} - total rows in this batch: {}".format(i, len(symbols), total_rows))

# #     if len(data) > 0:
# #         price_sdf = spark.createDataFrame(data)
# #         price_sdf.write.mode('append').format('csv').save(DB_HOST+TABLE_STOCK_PRICES, header=True)


# logger.warn("done!")

### Test

In [63]:
pdf_temp = spark.read.csv(DB_HOST+TABLE_STOCK_PRICES+'-temp', header=True, ignoreLeadingWhiteSpace=True, inferSchema=True).toPandas()
print(pdf_temp.info())
pdf_temp.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 12 columns):
date        93 non-null object
open        93 non-null object
high        93 non-null object
low         93 non-null object
close       93 non-null object
volume      93 non-null object
changed     93 non-null object
changep     93 non-null object
adjclose    93 non-null object
tradeval    93 non-null object
tradevol    93 non-null object
symbol      93 non-null object
dtypes: object(12)
memory usage: 8.8+ KB
None


,date,open,high,low,close,volume,changed,changep,adjclose,tradeval,tradevol,symbol
0,2019-12-12,272.09,272.575,270.20,270.29,1779499,-1.86,-0.68%,270.29,482215933.19,22880,SPGI
1,2019-12-12,166.73,170.85,166.325,168.56,2872410,-0.18,-0.11%,168.56,484265329.49,30864,MMM
2,2019-12-12,116.56,117.27,113.92,114.31,375268,-1.666,-1.44%,114.0482,43052432.92,8276,RGLD
3,2019-12-12,91.41,91.41,90.91,91.08,2281518,-0.289,-0.32%,90.8252,207835625.98,10173,VCIT
4,2019-12-12,109.60,112.84,109.15,112.47,1818087,2.97,2.71%,112.47,203237156.99,22552,ALXN


In [64]:
pdf = spark.read.csv(DB_HOST+TABLE_STOCK_PRICES, header=True, inferSchema=True) \
    .dropDuplicates(['date', 'symbol']).toPandas()
print(pdf.info())
pdf.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10549 entries, 0 to 10548
Data columns (total 12 columns):
date        10548 non-null datetime64[ns]
open        10548 non-null object
high        10548 non-null object
low         10548 non-null object
close       10548 non-null float64
volume      10548 non-null float64
changed     10548 non-null float64
changep     10548 non-null object
adjclose    10548 non-null float64
tradeval    10548 non-null object
tradevol    10548 non-null float64
symbol      10548 non-null object
dtypes: datetime64[ns](1), float64(5), object(6)
memory usage: 989.0+ KB
None


,date,open,high,low,close,volume,changed,changep,adjclose,tradeval,tradevol,symbol
0,2013-04-19,25.62,25.96,25.33,25.73,20353547.0,0.040,0.16%,25.7300,523595607.18,66068.0,FB
1,2013-04-22,99.35,99.66,98.38,99.32,5609009.0,-0.493,-0.60%,81.5900,555207178.14,32582.0,MCD
2,2013-04-26,100.83,100.99,100.40,100.89,3113532.0,-0.041,-0.05%,82.8798,313733616.64,16412.0,MCD
3,2013-05-13,26.60,27.325,26.531,26.82,29009648.0,0.140,0.52%,26.8200,782579320.46,90803.0,FB
4,2013-07-10,291.41,293.34,289.40,292.33,1822877.0,0.800,0.27%,292.3300,531832345.31,11812.0,AMZN


In [65]:
pdf.sort_values(by=['date', 'symbol'], ascending=True).head(5)

,date,open,high,low,close,volume,changed,changep,adjclose,tradeval,tradevol,symbol
5275,2013-04-02,262.40,265.89,260.55,263.322,2631038.0,1.712,0.65%,263.3220,693325169.20,17760.0,AMZN
2003,2013-04-02,25.77,26.12,25.30,25.420,35124893.0,-0.110,-0.43%,25.4200,904220577.12,107077.0,FB
2059,2013-04-02,99.40,100.42,99.025,100.260,5136501.0,0.994,1.22%,82.3622,513202156.02,24959.0,MCD
9859,2013-04-02,183.90,185.1799,176.10,176.690,4610979.0,-0.820,-3.15%,25.2414,828031668.71,27837.0,NFLX
5482,2013-04-02,43.60,45.50,43.5101,44.340,6621439.0,0.410,0.93%,44.3400,294906438.13,28077.0,TSLA


### Quality-check

In [66]:
# Pass into `args` argument

STOCKS = ['FB', 'GOOG', 'AMZN', 'TRMT', 'TSLA', 'MCD', 'NFLX']
DB_HOST = ''

# Table names: update to add '/' in the final code.
TABLE_STOCK_INFO_NASDAQ = 'test_data/raw/stock_info_nasdaq'
TABLE_STOCK_INFO_NYSE = 'test_data/raw/stock_info_nyse'
TABLE_STOCK_PRICES = 'test_data/raw/prices'

In [67]:
if STOCKS is None or len(STOCKS) == 0:
    check_basic_quality(logger, DB_HOST, TABLE_STOCK_INFO_NASDAQ)
    check_basic_quality(logger, DB_HOST, TABLE_STOCK_INFO_NYSE)
check_basic_quality(logger, DB_HOST, TABLE_STOCK_PRICES, table_type='csv')

WARN:root:(SUCCESS) Table test_data/raw/prices has 10654 rows.


## 6. Combine Datasets

### Code

In [68]:
# Pass into `args` argument

YESTERDAY_DATE = '2019-12-12'
AWS_ACCESS_KEY_ID = config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['AWS']['AWS_SECRET_ACCESS_KEY']
DB_HOST = ''

# Table names: update to add '/' in the final code.
TABLE_STOCK_PRICES = 'test_data/raw/prices'
TABLE_SHORT_INTERESTS_NASDAQ = 'test_data/raw/short_interests_nasdaq' 
TABLE_SHORT_INTERESTS_NYSE = 'test_data/raw/short_interests_nyse'
TABLE_SHORT_ANALYSIS = 'test_data/processed/short_analysis'

In [77]:
create_table = not(spark_table_exists(DB_HOST, TABLE_SHORT_ANALYSIS))

sdf_shorts = spark.read.csv([DB_HOST+TABLE_SHORT_INTERESTS_NASDAQ, DB_HOST+TABLE_SHORT_INTERESTS_NYSE], header=True, inferSchema=True)
sdf_shorts = sdf_shorts.withColumn('Date', sdf_shorts['Date'].cast(T.DateType()))

sdf_shorts = sdf_shorts.groupby(['Date', 'Symbol']) \
                 .agg(F.sum(sdf_shorts['ShortExemptVolume']).alias('short_exempt_volume'),
                      F.sum(sdf_shorts['ShortVolume']).alias('short_volume'),
                      F.sum(sdf_shorts['TotalVolume']).alias('total_volume')
                     )
sdf_prices = spark.read.csv(DB_HOST+TABLE_STOCK_PRICES, header=True, inferSchema=True) \
             .dropDuplicates(['date', 'symbol'])
sdf_prices = sdf_prices.withColumn('date', sdf_prices['date'].cast(T.DateType()))

if create_table == False:
    sdf_shorts = sdf_shorts.filter(sdf_shorts['Date'] >= F.to_date(F.lit(YESTERDAY_DATE)))
    sdf_prices = sdf_prices.filter(sdf_prices['date'] >= F.to_date(F.lit(YESTERDAY_DATE)))

sdf_short_analysis = sdf_shorts.join(sdf_prices, (sdf_shorts['Date'] == sdf_prices['date']) & \
                                     (sdf_shorts['Symbol'] == sdf_prices['symbol']), how='inner') \
                               .drop(sdf_shorts['Date']).drop(sdf_shorts['Symbol'])

mode = 'overwrite'
if create_table == False:
    logger.warn("Appending to table {}".format(DB_HOST+TABLE_SHORT_ANALYSIS))
    mode = 'append'
else:
    logger.warn("Creating table {}".format(DB_HOST+TABLE_SHORT_ANALYSIS))

# Coalesce is a must to avoid multiple headers.
sdf_short_analysis.coalesce(1).write.mode(mode).csv(DB_HOST+TABLE_SHORT_ANALYSIS, header=True)

delete_path(spark, DB_HOST, TABLE_SHORT_ANALYSIS+".csv")
copyMerge(spark, DB_HOST, DB_HOST+TABLE_SHORT_ANALYSIS, DB_HOST+TABLE_SHORT_ANALYSIS+".csv")

logger.warn("done!")

WARN:root:Creating table test_data/processed/short_analysis
WARN:root:done!


### Test

In [76]:
df_short = sdf_shorts.toPandas()
df_prices = sdf_prices.toPandas()
df_short_analysis = sdf_short_analysis.toPandas()

print(df_short.info())
print(df_prices.info())
print(df_short_analysis.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940084 entries, 0 to 940083
Data columns (total 5 columns):
Date                   940084 non-null object
Symbol                 940084 non-null object
short_exempt_volume    940084 non-null float64
short_volume           940084 non-null float64
total_volume           940084 non-null float64
dtypes: float64(3), object(2)
memory usage: 35.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10549 entries, 0 to 10548
Data columns (total 12 columns):
date        10548 non-null object
open        10548 non-null object
high        10548 non-null object
low         10548 non-null object
close       10548 non-null float64
volume      10548 non-null float64
changed     10548 non-null float64
changep     10548 non-null object
adjclose    10548 non-null float64
tradeval    10548 non-null object
tradevol    10548 non-null float64
symbol      10548 non-null object
dtypes: float64(5), object(7)
memory usage: 989.0+ KB
None
<class 'pandas.cor

In [19]:
spark.read.parquet(DB_HOST+TABLE_SHORT_ANALYSIS).toPandas().head(5)

,short_exempt_volume,short_volume,total_volume,source_url,date,open,high,low,close,volume,changed,changep,adjclose,tradeval,tradevol,symbol
0,0.0,418961.0,753385.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,2013-05-01,215.92,217.389,211.65,212.91,2622654,-0.451,-1.46%,30.4157,561027949.42,14673,NFLX
1,37360.0,6060485.0,20502608.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,2013-05-06,28.33,28.46,27.48,27.57,43862625,-0.741,-2.62%,27.5700,1219158766.94,120016,FB
2,0.0,635427.0,1388246.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,2013-05-06,209.63,212.45,204.02,210.69,4532918,-0.394,-1.29%,30.0985,949314738.05,26760,NFLX
3,300.0,451077.0,952404.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,2013-05-09,258.73,263.55,256.88,260.16,2769255,1.480,0.57%,260.1600,723085024.78,17822,AMZN
4,0.0,342728.0,1335921.0,https://www.quandl.com/api/v3/datasets/FINRA/F...,2013-06-28,276.19,279.83,276.19,277.69,3193262,0.140,0.05%,277.6900,889020787.27,14332,AMZN


In [24]:
spark.read.parquet(DB_HOST+TABLE_SHORT_ANALYSIS).repartition(1).write.mode('overwrite').format('csv').save(DB_HOST+TABLE_SHORT_ANALYSIS+".csv", header=True)

### Quality-check

In [73]:
# Pass into `args` argument

DB_HOST = ''

# Table names: update to add '/' in the final code.
TABLE_SHORT_ANALYSIS = 'test_data/processed/short_analysis'

In [74]:
check_basic_quality(logger, DB_HOST, TABLE_SHORT_ANALYSIS)

WARN:root:(SUCCESS) Table test_data/processed/short_analysis has 19087 rows.


## Final Dataset

In [11]:
df = sdf.toPandas()

In [12]:
df.describe()

,short_exempt_volume,short_volume,total_volume,close,volume,changed,adjclose,tradevol
count,1.054800e+04,1.054800e+04,1.054800e+04,10548.000000,1.054800e+04,10548.000000,10548.000000,1.054800e+04
mean,1.186437e+04,1.403028e+06,3.208060e+06,394.143509,9.086290e+06,0.389712,374.410873,6.666282e+04
std,4.619000e+04,2.289725e+06,5.337051e+06,422.500286,1.448612e+07,11.274337,429.671097,6.752694e+04
min,0.000000e+00,1.000000e+00,1.000000e+00,3.940000,5.300000e+01,-139.360000,3.612400,0.000000e+00
25%,5.000000e+02,2.772368e+05,6.956500e+05,120.627500,2.423723e+06,-1.401000,99.175000,2.628525e+04
50%,2.183000e+03,6.329955e+05,1.455506e+06,221.885000,4.423890e+06,0.080000,196.520200,4.293400e+04
75%,8.187750e+03,1.543929e+06,3.499728e+06,496.172500,9.444361e+06,2.089250,428.165000,8.723925e+04
max,1.653923e+06,4.887740e+07,1.201625e+08,2039.510000,3.653806e+08,558.460000,2039.510000,1.312878e+06


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10548 entries, 0 to 10547
Data columns (total 14 columns):
short_exempt_volume    10548 non-null float64
short_volume           10548 non-null float64
total_volume           10548 non-null float64
source_url             10548 non-null object
open                   10548 non-null object
high                   10548 non-null object
low                    10548 non-null object
close                  10548 non-null float64
volume                 10548 non-null int32
changed                10548 non-null float64
changep                10548 non-null object
adjclose               10548 non-null float64
tradeval               10548 non-null object
tradevol               10548 non-null int32
dtypes: float64(6), int32(2), object(6)
memory usage: 1.0+ MB


## References

- Best practices: https://towardsdatascience.com/apache-airflow-tips-and-best-practices-ff64ce92ef8
- Write/store dataframe as textfile: https://stackoverflow.com/questions/44537889/write-store-dataframe-in-text-file
- Delete hdfs path: https://stackoverflow.com/a/55952480/278191
- Delete hdfs path in S3: http://bigdatatech.taleia.software/2015/12/28/deleting-a-amazon-s3-path-from-apache-spark/
- Import java class in python: https://stackoverflow.com/questions/33544105/running-custom-java-class-in-pyspark